### Import Required Libraries and Set Up Environment Variables

In [63]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [64]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [65]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL



In [66]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    query = {
        'api-key': nyt_api_key,
        'q': filter_query,
        'sort': sort,
        'fl': field_list,
        'begin_date': begin_date,
        'end_date': end_date,
        'page': page
    }

    # Make a "GET" request and retrieve the JSON
    response = requests.get(url, params=query)
    reviews = response.json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        if reviews["response"]["docs"]:
            # loop through the reviews["response"]["docs"] and append each review to the list
            for review in reviews["response"]["docs"]:
                reviews_list.append(review)
            # Print the page that was just retrieved
            print(f"Retrieved page {page}")
        else:
            # Print the page number that had no results then break from the loop
            print(f"No results on page {page}, stopping.")
            break
    except KeyError:
        print(f"Error on page {page}, skipping.")




Error on page 0, skipping.
Error on page 1, skipping.
Error on page 2, skipping.
Error on page 3, skipping.
Error on page 4, skipping.
Error on page 5, skipping.
Error on page 6, skipping.
Error on page 7, skipping.
Error on page 8, skipping.
Error on page 9, skipping.
Error on page 10, skipping.
Error on page 11, skipping.
Error on page 12, skipping.
Error on page 13, skipping.
Error on page 14, skipping.
Error on page 15, skipping.
Error on page 16, skipping.
Error on page 17, skipping.
Error on page 18, skipping.
Error on page 19, skipping.


In [67]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[]


In [52]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = pd.json_normalize(reviews_list)

In [53]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].str.extract('\u2018(.*?)\u2019 Review')


KeyError: 'headline.main'

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)


In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df['title'].to_list()

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(60)

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    search_url = url + title + tmdb_key_string
    response = requests.get(search_url)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        movie_id = response.json()['results'][0]['id']

        # Get movie id
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Make a request for the full movie details
        movie_response = requests.get(movie_url)
        movie_details = movie_response.json()

        # Execute "GET" request with url
        genres = [genre['name'] for genre in movie_details['genres']]
        languages = [lang['english_name'] for lang in movie_details['spoken_languages']]
        countries = [country['name'] for country in movie_details['production_countries']]

        # Extract the genre names into a list
        # Extract the spoken_languages' English name into a list
        # Extract the production_countries' name into a list

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            'title': title,
            'genres': genres,
            'languages': languages,
            'countries': countries
        })

        # Print out the title that was found
        print(f"Found: {title}")

    except IndexError:
        print(f"No TMDB entry found for: {title}")



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, df_tmdb, on='title')


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
list_columns = ['genres', 'languages', 'countries']

# Create a list of characters to remove
chars_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for col in list_columns:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df)


In [ ]:
# Drop "byline.person" column
merged_df.drop(columns=['byline.person'], inplace=True)


In [ ]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)


In [ ]:
# Export data to CSV without the index
merged_df.to_csv('movie_reviews_data.csv', index=False)

